# 1) Sample the interesting galaxies

# Overview
1) The "What"
2) Very small intermezzo on hydrosimulations
3) Loose structure of the code
4) The "how" - tutorial of the code

# The "What"
We want to study gravitational lenses obtained from particles of hydrosimulated galaxies- even more precisely, we want 

1) find lenses
2) compute the $\alpha$ map
3) study their density

While our project will differs in many aspects, these 3 core point should more or less be in common (except some possible different parametric requirement, e.g. the type of lenses).

# Very small intermezzo on hydrosimulations
You probably know as well as me, if not better, what an hydrodynamical simulation (henceforth hydrosim) is. However, just to be sure we all on the same page, I will recap here some very basic but necessary informations.
Hydrosims are composed of particles, divided in 4 classes:
1) **dark matter**
2) **gas**
3) **stars**
4) **black holes**


They mainly interact gravitationally, but being hydro (and not n-body) simulations, they also have peculiar and individual properties: star-formation, feedbacks, accretion etc. For what concerns lensing, however, we luckily can ignore all this and just treat them as massive particles for our lenses. 

Furthermore, the simulations evolve in time (thus in redshifts z), and its properties are then stored at given **snapshots (snap**) in time. Thus **z $\leftrightarrow$ snap**.

In each of these, the galaxy catalogues have been compiled (luckily!), thus finding the formed galaxies and linking the particle to a given galaxy, which in turn is charaterised by Mass and its redshift, properties we can select for.

# How

<figure>
<img src="tutorial_images/how.png" style="width:100%">
<figcaption align = "center"> 
Outline of pipeline aim
</figcaption>
</figure>


# Loose structure of the code



Finding lenses is the easiest part, given a galaxy catalogue (which is the case for EAGLE and COLIBRE luckily). We define a range of redshifts and a minimum mass, and get a list of galaxies that are our lens candidates. 

From this list, we can sample a candidate lens, and load its particles from the simulation dataset. We only care about their coordinates and masses, not their type.

We can then project these particles into a 2D mass distribution 

and verify if their density is supercritical - even better, we can define a minimum $\theta_E$ such that our lensed images have a minimum of separation and can be resolved:

<figure>
<img src="tutorial_images/Sig_enc.png" style="width:100%">
<figcaption align = "center"> 
    
The black curve is the enclosed surface density $\Sigma_{encl}$ within an aperture $\theta$

$\Sigma_{crit}^{min}=\Sigma_{crit}(z_{source\, max}=4)$ is the minimum critical density allowed for the galaxy to be considered a lens, i.e. the critical density if the source is at the maximum allowed redshift  $z_{source\, max}$ (by default  $z_{source\, max}=$4)

$\theta_{min}$ is the minum allowed $\theta_E$

$\Sigma(\theta_{min})$ is the surface density at $\theta_{min}$ for this galaxy

$\theta_E(z_{s,max})$ is the Einstein radius of this lens if the source is located at the maximum allowed redshift $z_{source\, max}$.


$\Sigma_{crit}$ is the critical density given the sampled source redshift

$\theta_E$ is the resulting Einstein radius for this lens.
</figcaption>
</figure>
Note that for this we have to define a range of source redshift $z_{\text{source}}$, notably the maximum allowed $z_{\text{source max}}$, since the min is given by the lens redshift. 

To be even more picky, we defined a minimum Einstein radius $\theta_{min}$, such that the lensed images are separated by a minimum distance/angle, which is then compatible with realistic observational capabilities.

Note also that these computation of Einstein angle are approximate, since we are reducing the 2D density distribution into 1D, so it is appropriate only as a rough estimate of the quality of the lens.

